PHY5340 Laboratory 6: Finite Differences

Jeremiah O'Neil, SN6498391

---

# Finite Differences for Eigenvalues and Eigenfunctions

In [ ]:
newV = @(pars) (@(x) pars(1)*x.^2 + pars(2)*x.^3 + pars(3)*x.^4);

In [ ]:
%type('tise_fd.m')
V = newV([50, 0, 2500]);
[eigE, eigfuncs, x] = tise_fd(V, -0.5, 0.5, 2000, 100, 3);

In [ ]:
%type('plot_eigs.m')
plot_eigs(x, V, eigE, eigfuncs)
xlim([-0.5, 0.5])
ylim([0, 20])

In [ ]:
V = newV([50, 1500, 2500]);
[eigE, eigfuncs, x] = tise_fd(V, -0.9, 0.5, 2000, 100, 3);
plot_eigs(x, V, eigE, eigfuncs)
xlim([-0.8, 0.4]);
ylim([-25, 5]);

# Finite Differences for Nonlinear ODE

In [ ]:
%type('thomas_fermi_fd.m')
r = linspace(0, 100, 101);
n_init = linspace(1, 0, 101);
plot(r, n_init); hold on;
n = thomas_fermi_fd(n_init, r, 0, 1e-5, 10000);
plot(r, n); hold off;

In [ ]:
N = 20;
numiter = zeros(1, N);
# want alpha to be N evenly spaced values in [0,1)
# this sort of thing is why FORTRAN indexing sucks
alpha = linspace(0, 1, N+1);
alpha = alpha(1:end-1);
for i = [1:N]
    [~, numiter(i)] = thomas_fermi_fd(n_init, r, alpha(i), 1e-5, 10000);
end
plot(alpha, numiter)